In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from termcolor import colored
import urllib.request
import os
import time
import uuid

In [2]:
service = Service("chromedriver_win64\chromedriver.exe")
options = Options()
# options.add_argument("--headless")
options.add_argument("--window-size=1920,1080")
options.add_argument('--ignore-certificate-errors')
options.add_argument('--allow-running-insecure-content')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                        "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36")
driver = webdriver.Chrome(service=service, options=options)
    

In [3]:
from dotenv import load_dotenv
load_dotenv()
usr = os.getenv("USR")
pswd = os.getenv("PSWD")

In [4]:
login_page = "https://www.instagram.com/accounts/login/"
driver.get(login_page)
time.sleep(4)

# Login with a random account since we can't scrap stories without being logged
driver.find_element(By.NAME, "username").send_keys(usr)
driver.find_element(By.NAME, "password").send_keys(pswd)

# Click the login button
driver.find_element(By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div').click()
print(colored("\n[SUCCESS]: Logged into the website. \n", "green"))
time.sleep(5)


[SUCCESS]: Logged into the website. 



In [5]:
# Click the "Not Now" button for the save login info
buttons = driver.find_elements(By.CSS_SELECTOR, 'div[role="button"]')
for button in buttons:
    # print(button.get_attribute("innerHTML"))
    if button.get_attribute("innerHTML") == "Not now":
        button.click()
        print(colored("\n[SUCCESS]: Clicked the 'Not Now' button. \n", "green"))
        break
time.sleep(5)


[SUCCESS]: Clicked the 'Not Now' button. 



In [6]:
# Click the "Not Now" button for the notifications
buttons = driver.find_elements(By.TAG_NAME, 'button')
for button in buttons:
    if button.get_attribute("innerHTML") == "Not Now":
        button.click()
        print(colored("\n[SUCCESS]: Clicked the 'Not Now' button. \n", "green"))
        break
time.sleep(5)


[SUCCESS]: Clicked the 'Not Now' button. 



In [11]:
story_items = driver.find_elements(By.CSS_SELECTOR, 'button[role="menuitem"]')
# for story_item in story_items:
#     print(story_item.get_attribute("innerHTML"))
story_items[0].click()

In [12]:
rows = {}

while driver.current_url != "https://www.instagram.com/":
    
    # collect the username of the account
    username = driver.current_url.split("/")[4]
    
    is_video = False
    try:
        content_element = driver.find_element(By.XPATH, \
            '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/section/div[1]/div/div[5]/section/div/div[1]/div/div/img')
        content_link = content_element.get_attribute('src')
        
    except:
        is_video = True
        probable_elements = driver.find_elements(By.CSS_SELECTOR, 'video')
        for element in probable_elements:
            print(element.get_attribute('innerHTML'))
        content_link = probable_elements[0].get_attribute('src')
        
    print(colored(f"\n[SUCCESS]: Got the content link:\n\t{content_link}. \n", "green"))

    # Get the link of the story
    insta_link = driver.current_url

    # Get the date of the story
    date = driver.find_element(By.CSS_SELECTOR, 'time').get_attribute("datetime")

    # Append all collected information into a row
    rows[username] = rows.get(username, []) + \
    [
        {
            'Instagram URL': insta_link,
            'Content URL': content_link,
            'Date': date,
            'is_video': is_video
        }
    ]

    # Click on the next button
    try:
        buttons = driver.find_elements(By.CSS_SELECTOR, 'button')
        for button in buttons:
            if button.get_attribute('aria-label') == 'Next':
                button.click()
                break
    except:
        break

    time.sleep(1)
    
# Save scrapped data into a dataframe
tmp = os.path.join("collected_data", "pickles")
if not os.path.exists(tmp): os.makedirs(tmp)

for username in rows:
    df = pd.DataFrame(rows[username])
    df.to_pickle(os.path.join(tmp, "stories_{}.pkl".format(username)))
    # driver.quit()
    
print(colored("\n[SUCCESS]: Scrapped all stories for the last 24h. \n", "green"))



[SUCCESS]: Got the content link:
	blob:https://www.instagram.com/87642ae6-ebba-448e-8d9a-56b716228383. 


[SUCCESS]: Got the content link:
	https://instagram.fdac107-1.fna.fbcdn.net/v/t51.2885-15/425144600_904280281164341_3124477142352824091_n.jpg?stp=dst-jpg_e35&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi43NTB4MTMzNC5zZHIifQ&_nc_ht=instagram.fdac107-1.fna.fbcdn.net&_nc_cat=1&_nc_ohc=QtxqJkRiTnsAX9WXKC0&edm=ANmP7GQBAAAA&ccb=7-5&ig_cache_key=MzI5MzgwNTQ2MDQ3OTY1ODU2OQ%3D%3D.2-ccb7-5&oh=00_AfDvrVXmAjuvN5eKAxlnFmMs5ylTsJHgH8Z_PyfePbO-og&oe=65BF5F16&_nc_sid=982cc7. 


[SUCCESS]: Got the content link:
	https://instagram.fdac107-1.fna.fbcdn.net/v/t51.2885-15/424436735_7538970126136246_6944427874772735566_n.jpg?stp=dst-jpg_e35&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi43NTB4MTMzNC5zZHIifQ&_nc_ht=instagram.fdac107-1.fna.fbcdn.net&_nc_cat=1&_nc_ohc=BpFrg_puv1YAX8x3oZe&edm=ANmP7GQBAAAA&ccb=7-5&ig_cache_key=MzI5MzgwNTYxNTk1NDAyMjExNw%3D%3D.2-ccb7-5&oh=00_AfARe0HItq-3QGzvSAZBg4lYRq4nq7VxFvsFg2YFn

In [13]:
rows

{'realmadrid': [{'Instagram URL': 'https://www.instagram.com/stories/realmadrid/3293778007233269442/',
   'Content URL': 'blob:https://www.instagram.com/87642ae6-ebba-448e-8d9a-56b716228383',
   'Date': '2024-02-02T10:15:47.000Z',
   'is_video': True},
  {'Instagram URL': 'https://www.instagram.com/stories/realmadrid/3293805460479658569/',
   'Content URL': 'https://instagram.fdac107-1.fna.fbcdn.net/v/t51.2885-15/425144600_904280281164341_3124477142352824091_n.jpg?stp=dst-jpg_e35&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi43NTB4MTMzNC5zZHIifQ&_nc_ht=instagram.fdac107-1.fna.fbcdn.net&_nc_cat=1&_nc_ohc=QtxqJkRiTnsAX9WXKC0&edm=ANmP7GQBAAAA&ccb=7-5&ig_cache_key=MzI5MzgwNTQ2MDQ3OTY1ODU2OQ%3D%3D.2-ccb7-5&oh=00_AfDvrVXmAjuvN5eKAxlnFmMs5ylTsJHgH8Z_PyfePbO-og&oe=65BF5F16&_nc_sid=982cc7',
   'Date': '2024-02-02T11:10:27.000Z',
   'is_video': False},
  {'Instagram URL': 'https://www.instagram.com/stories/realmadrid/3293805615954022117/',
   'Content URL': 'https://instagram.fdac107-1.fna.fbcdn.net

In [14]:
for username in rows:
    for row in rows[username]:
        content_link = row["Content URL"]
        filename = f"{uuid.uuid4()}.jpg"
        dir = os.path.join("collected_data", username)
        if not os.path.exists(dir): os.makedirs(dir)
        try:
            urllib.request.urlretrieve(content_link, os.path.join(dir, filename))
            print(colored(f"\n[SUCCESS]: Downloaded the content link:\n\t{content_link}. \n", "green"))
            print(colored(f"\n[SUCCESS]: Saved the content to the file:\n\t{filename}. \n", "green"))
        except Exception as e:
            print(colored(f"\n[ERROR]: Couldn't download the content link:\n\t{content_link}. \n", "red"))
            print(e)


[ERROR]: Couldn't download the content link:
	blob:https://www.instagram.com/87642ae6-ebba-448e-8d9a-56b716228383. 

<urlopen error unknown url type: blob>

[SUCCESS]: Downloaded the content link:
	https://instagram.fdac107-1.fna.fbcdn.net/v/t51.2885-15/425144600_904280281164341_3124477142352824091_n.jpg?stp=dst-jpg_e35&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi43NTB4MTMzNC5zZHIifQ&_nc_ht=instagram.fdac107-1.fna.fbcdn.net&_nc_cat=1&_nc_ohc=QtxqJkRiTnsAX9WXKC0&edm=ANmP7GQBAAAA&ccb=7-5&ig_cache_key=MzI5MzgwNTQ2MDQ3OTY1ODU2OQ%3D%3D.2-ccb7-5&oh=00_AfDvrVXmAjuvN5eKAxlnFmMs5ylTsJHgH8Z_PyfePbO-og&oe=65BF5F16&_nc_sid=982cc7. 


[SUCCESS]: Saved the content to the file:
	99fc9c5a-bfc4-4d4c-a720-85d57114f785.jpg. 


[SUCCESS]: Downloaded the content link:
	https://instagram.fdac107-1.fna.fbcdn.net/v/t51.2885-15/424436735_7538970126136246_6944427874772735566_n.jpg?stp=dst-jpg_e35&efg=eyJ2ZW5jb2RlX3RhZyI6ImltYWdlX3VybGdlbi43NTB4MTMzNC5zZHIifQ&_nc_ht=instagram.fdac107-1.fna.fbcdn.net&_nc_cat=1&_nc